# MongoDB

In [2]:
import time
from pymongo import MongoClient


mongodb_url = 'mongodb://your-db-url'
client = MongoClient(mongodb_url)

db = client['your-db']

test_coll = db['test_coll'] #get collection

the_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
data1 =  {
  "H": 80, 
  "T": 30, 
  "time": the_time, 
  "uuid": "yang_home"
}

res = test_coll.insert(data1)
print(res)


59e41779183b030044d64fbd


/app/.conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [3]:
# 查資料
print('--> find all')
res = test_coll.find() #全抓
print(list(res))

--> find all
[{'_id': ObjectId('59e41779183b030044d64fbd'), 'H': 80, 'T': 30, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'}]


In [8]:
# insert multi-data

data =  [
    {"H": 90, "T": 32, "time": the_time, "uuid": "yang_home"},
    {"H": 85, "T": 31, "time": the_time, "uuid": "yang_home"}
]

res = test_coll.insert(data)
print('result: ', res)

print('----find data----')
# 查資料
print('--> find all')
cursor = test_coll.find() #全抓
data = list(cursor)
for item in data:
    item.pop('_id')
data


result:  [ObjectId('59e41935183b030044d64fc2'), ObjectId('59e41935183b030044d64fc3')]
----find data----
--> find all


/app/.conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[{'H': 80, 'T': 30, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 90, 'T': 32, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 85, 'T': 31, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 90, 'T': 32, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 85, 'T': 31, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 90, 'T': 32, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'},
 {'H': 85, 'T': 31, 'time': '2017-10-16 02:20:41', 'uuid': 'yang_home'}]

In [9]:
import pandas as pd
cursor = test_coll.find() #全抓

df = pd.DataFrame(list(cursor))
df

,H,T,_id,time,uuid
0,80,30,59e41779183b030044d64fbd,2017-10-16 02:20:41,yang_home
1,90,32,59e4185d183b030044d64fbe,2017-10-16 02:20:41,yang_home
2,85,31,59e4185d183b030044d64fbf,2017-10-16 02:20:41,yang_home
3,90,32,59e418dd183b030044d64fc0,2017-10-16 02:20:41,yang_home
4,85,31,59e418dd183b030044d64fc1,2017-10-16 02:20:41,yang_home
5,90,32,59e41935183b030044d64fc2,2017-10-16 02:20:41,yang_home
6,85,31,59e41935183b030044d64fc3,2017-10-16 02:20:41,yang_home


### 當然，也可以設條件來查詢
也很簡單的以dict給條件即可 --> mongoDB都是使用JSON(or say BSON)，所以實質上也許是轉為JSON進一步處理也說不一定

In [ ]:
# 查資料
print('--> find all')
cursor = test_coll.find({'uuid': yang_home}) #找出'uuid'為'yang_home'的
data = list(cursor)
for item in data:
    item.pop('_id')
data

In [ ]:
# 也有大於，小於的寫法，但要用mongoDB的自定義符號
#大於
test_coll.find({'time':{'$gt':'2017-10-16 02:20:41'}})
#大於等於
test_coll.find({'time':{'$gte':'2017-10-16 02:20:41'}})
#小於
test_coll.find({'time':{'$lt':'2017-10-16 02:20:41'}})
#小於等於
test_coll.find({'time':{'$lte':'2017-10-16 02:20:41'}})


# PostgreSQL

In [10]:
pg_db_url = 'postgres://your-db-url'

In [11]:
from sqlalchemy import create_engine
pg_db = create_engine(pg_db_url)

In [12]:
# check and create table
resultProxy = pg_db.execute("CREATE TABLE IF NOT EXISTS test_table(uuid text NOT NULL, time text NOT NULL, data text)")


In [18]:
import json
import time

# PostgreSQL insert
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
uuid = 'yang_home'
data = {"H": 80, "T": 30}
data = json.dumps(data)
resultProxy=pg_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [20]:
resultProxy.__dict__

{'_echo': False,
 '_saved_cursor': <cursor object at 0x7fce62a09330; closed: -1>,
 '_soft_closed': True,
 'connection': <sqlalchemy.engine.base.Connection at 0x7fce16caf3c8>,
 'context': <sqlalchemy.dialects.postgresql.psycopg2.PGExecutionContext_psycopg2 at 0x7fce16caf7b8>,
 'cursor': None,
 'dialect': <sqlalchemy.dialects.postgresql.psycopg2.PGDialect_psycopg2 at 0x7fce17caa2e8>,
 'rowcount': 1}

In [17]:
# PostreSQL select
resultProxy=pg_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('yang_home', '2017-10-16 03:01:52', '{"H": 90, "T": 32}')]


In [21]:
# insert multi-data
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
time.sleep(1)
my_time2 = time.strftime("%Y-%m-%d %H:%M:%S")

data_list =  [
    {'data': {"H": 90, "T": 32}
         , "time": my_time
         , "uuid": "yang_home"},
    {'data': {"H": 80, "T": 30}
         , "time": my_time2
         , "uuid": "yang_home"},
]

for item in data_list:
    uuid = item['uuid']
    my_time = item['time']
    data = item['data']
    data = json.dumps(data)
    resultProxy=pg_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [35]:
# PostreSQL select
resultProxy=pg_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('yang_home', '2017-10-16 03:01:52', '{"H": 90, "T": 32}'), ('yang_home', '2017-10-16 03:11:22', '{"H": 80, "T": 30}'), ('yang_home', '2017-10-16 03:12:50', '{"H": 90, "T": 32}'), ('yang_home', '2017-10-16 03:12:51', '{"H": 80, "T": 30}')]


In [36]:
import pandas as pd

df = pd.DataFrame(data)
df

,0,1,2
0,yang_home,2017-10-16 03:01:52,"{""H"": 90, ""T"": 32}"
1,yang_home,2017-10-16 03:11:22,"{""H"": 80, ""T"": 30}"
2,yang_home,2017-10-16 03:12:50,"{""H"": 90, ""T"": 32}"
3,yang_home,2017-10-16 03:12:51,"{""H"": 80, ""T"": 30}"


# MySQL
ref: http://mookrs.com/flask-sqlalchemy-mysql-python-3/


In [24]:
mysql_db_url = 'mysql://your-db-url'

In [27]:
# 先裝 mysql的connection driver
# pip install PyMySQL

from sqlalchemy import create_engine
# add "+pymysql" in url
mysql_db_url = 'mysql+pymysql://your-db-url'
my_db = create_engine(mysql_db_url)


In [28]:
# check and create table
resultProxy = my_db.execute("CREATE TABLE IF NOT EXISTS test_table(uuid text NOT NULL, time text NOT NULL, data text)")
resultProxy.__dict__

{'_echo': False,
 '_saved_cursor': <pymysql.cursors.Cursor at 0x7fce165739e8>,
 '_soft_closed': True,
 'connection': <sqlalchemy.engine.base.Connection at 0x7fce16573c50>,
 'context': <sqlalchemy.dialects.mysql.mysqldb.MySQLExecutionContext_mysqldb at 0x7fce16573320>,
 'cursor': None,
 'dialect': <sqlalchemy.dialects.mysql.pymysql.MySQLDialect_pymysql at 0x7fce1659d160>,
 'rowcount': 0}

In [29]:
import json
import time

# PostgreSQL insert
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
uuid = 'yang_home'
data = {"H": 80, "T": 30}
data = json.dumps(data)
resultProxy=my_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [30]:
resultProxy.__dict__

{'_echo': False,
 '_saved_cursor': <pymysql.cursors.Cursor at 0x7fce16515320>,
 '_soft_closed': True,
 'connection': <sqlalchemy.engine.base.Connection at 0x7fce16613b38>,
 'context': <sqlalchemy.dialects.mysql.mysqldb.MySQLExecutionContext_mysqldb at 0x7fce165152e8>,
 'cursor': None,
 'dialect': <sqlalchemy.dialects.mysql.pymysql.MySQLDialect_pymysql at 0x7fce1659d160>,
 'rowcount': 1}

In [31]:
# select
resultProxy=my_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('yang_home', '2017-10-16 03:32:17', '{"H": 80, "T": 30}')]


In [32]:
# insert multi-data
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
time.sleep(1)
my_time2 = time.strftime("%Y-%m-%d %H:%M:%S")

data_list =  [
    {'data': {"H": 90, "T": 32}
         , "time": my_time
         , "uuid": "yang_home"},
    {'data': {"H": 80, "T": 30}
         , "time": my_time2
         , "uuid": "yang_home"},
]

for item in data_list:
    uuid = item['uuid']
    my_time = item['time']
    data = item['data']
    data = json.dumps(data)
    resultProxy=my_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [37]:
# select
resultProxy=my_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('yang_home', '2017-10-16 03:32:17', '{"H": 80, "T": 30}'), ('yang_home', '2017-10-16 03:32:53', '{"H": 90, "T": 32}'), ('yang_home', '2017-10-16 03:32:54', '{"H": 80, "T": 30}')]


In [38]:
import pandas as pd

df = pd.DataFrame(data)
df

,0,1,2
0,yang_home,2017-10-16 03:32:17,"{""H"": 80, ""T"": 30}"
1,yang_home,2017-10-16 03:32:53,"{""H"": 90, ""T"": 32}"
2,yang_home,2017-10-16 03:32:54,"{""H"": 80, ""T"": 30}"


# MariaDB
### 其實是和mysql 共用driver的

In [39]:
#maria db url
db_url = 'mysql://your-db-url'

In [40]:
# 先裝 mysql的connection driver
# pip install PyMySQL

from sqlalchemy import create_engine
# add "+pymysql" in url
db_url = 'mysql+pymysql://your-db-url'

my_db = create_engine(db_url)


In [41]:
# check and create table
resultProxy = my_db.execute("CREATE TABLE IF NOT EXISTS test_table(uuid text NOT NULL, time text NOT NULL, data text)")
resultProxy.__dict__

{'_echo': False,
 '_saved_cursor': <pymysql.cursors.Cursor at 0x7fce16523438>,
 '_soft_closed': True,
 'connection': <sqlalchemy.engine.base.Connection at 0x7fce16573a58>,
 'context': <sqlalchemy.dialects.mysql.mysqldb.MySQLExecutionContext_mysqldb at 0x7fce16523710>,
 'cursor': None,
 'dialect': <sqlalchemy.dialects.mysql.pymysql.MySQLDialect_pymysql at 0x7fce16515cc0>,
 'rowcount': 0}

In [42]:
import json
import time

# PostgreSQL insert
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
uuid = 'yang_home'
data = {"H": 80, "T": 30}
data = json.dumps(data)
resultProxy=my_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [43]:
resultProxy.__dict__

{'_echo': False,
 '_saved_cursor': <pymysql.cursors.Cursor at 0x7fce16523b70>,
 '_soft_closed': True,
 'connection': <sqlalchemy.engine.base.Connection at 0x7fce4cf2b898>,
 'context': <sqlalchemy.dialects.mysql.mysqldb.MySQLExecutionContext_mysqldb at 0x7fce16523b38>,
 'cursor': None,
 'dialect': <sqlalchemy.dialects.mysql.pymysql.MySQLDialect_pymysql at 0x7fce16515cc0>,
 'rowcount': 1}

In [44]:
# select
resultProxy=my_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('yang_home', '2017-10-16 03:55:41', '{"H": 80, "T": 30}')]


In [45]:
# insert multi-data
my_time = time.strftime("%Y-%m-%d %H:%M:%S")
time.sleep(1)
my_time2 = time.strftime("%Y-%m-%d %H:%M:%S")

data_list =  [
    {'data': {"H": 90, "T": 32}
         , "time": my_time
         , "uuid": "yang_home"},
    {'data': {"H": 80, "T": 30}
         , "time": my_time2
         , "uuid": "yang_home"},
]

for item in data_list:
    uuid = item['uuid']
    my_time = item['time']
    data = item['data']
    data = json.dumps(data)
    resultProxy=my_db.execute("insert into test_table (uuid, time, data) values('%s', '%s', '%s')" %(uuid, my_time, data))

In [46]:
# select
resultProxy=my_db.execute("select * from test_table")
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('yang_home', '2017-10-16 03:55:41', '{"H": 80, "T": 30}'), ('yang_home', '2017-10-16 03:55:53', '{"H": 90, "T": 32}'), ('yang_home', '2017-10-16 03:55:54', '{"H": 80, "T": 30}')]


In [47]:
import pandas as pd

df = pd.DataFrame(data)
df

,0,1,2
0,yang_home,2017-10-16 03:55:41,"{""H"": 80, ""T"": 30}"
1,yang_home,2017-10-16 03:55:53,"{""H"": 90, ""T"": 32}"
2,yang_home,2017-10-16 03:55:54,"{""H"": 80, ""T"": 30}"
